In [4]:
import math
import time
import networkx as nx
import ipywidgets as widgets
import pandas as pd
from ipywidgets import interact, interact_manual
from pyvis.network import Network
from collections import deque

def read_matrix(file):
    matrix = []
    with open(file, "r") as f:
        for line in f.readlines():
            matrix.append(line.split())
    matrix = [[int(num) for num in matrix[i]] for i in range(len(matrix))]
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] == 0 and i != j:
                matrix[i][j] = math.inf
    return matrix

def make_graph(matrix, m_type):
    G = nx.MultiDiGraph()
    if m_type == 0:
        for i in range(len(matrix[0])):
            x = -1
            for j in range(len(matrix)):
                if x != -1 and matrix[j][i] != 0 and matrix[j][i] != math.inf:
                    G.add_edge(x, j+1, color='red')
                if x == -1 and matrix[j][i] != 0 and matrix[j][i] != math.inf:
                    x = j+1
    if m_type == 1:
        for i in range(len(matrix)):
            G.add_node(i+1, color='red')
            for j in range(len(matrix[0])):
                if i != j and matrix[i][j] != math.inf:
                    G.add_edge(j+1, i+1, weight=matrix[i][j], color='red')
    nt = Network(notebook=True, directed=True)
    nt.from_nx(G)
    return nt

def user_make_graph(add_node="", add_edge="", edge_width="1"):
    if add_node != "":
        add_node = add_node.split()
        graph.add_node(int(add_node[0]), color='red')
    if add_edge != "":
        if "," in add_edge and edge_width != "":
            graph.add_edge(int(add_edge.split(",")[0]), int(add_edge.split(",")[1]), width=edge_width, weight=edge_width, color='red')
    nb.from_nx(graph)
    return nb.show("basic.html")

def make_adjacency_matrix(G):
    matrix = [[0]*len(G.nodes) for i in range(len(G.nodes))]
    for i in range(1,len(G.nodes)):
        pairs = list(G.edges)
        for j in range(len(pairs)):
            if pairs[j]['from'] == i:
                matrix[pairs[j]['to']-1][i-1] = pairs[j]['weight']
            if pairs[j]['to'] == i:
                matrix[i-1][pairs[j]['from']-1] = pairs[j]['weight']
    return matrix

def new_graph(G, res):
    graph = nx.MultiDiGraph()
    edges = list(G.edges)
    for i in range(len(edges)):
        if not edges[i]['from'] in res or not edges[i]['to'] in res:
            graph.add_edge(edges[i]['from'], edges[i]['to'], weight=edges[i]['weight'], width=1, color="red")
    for i in range(len(edges)):
        if edges[i]['from'] in res and edges[i]['to'] in res:
            graph.add_edge(edges[i]['from'], edges[i]['to'], weight=edges[i]['weight'], width=3, color="green")
    nt = Network(notebook=True, directed=True)
    nt.from_nx(graph)
    return nt

def remove_edge(G, edge):
    M = make_adjacency_matrix(G)
    M[edge[1]-1][edge[0]-1] = 0
    for i in range(len(M)):
        for j in range(len(M[i])):
            if M[i][j] == 0 and i != j:
                M[i][j] = math.inf
    G = make_graph(M, 1)
    return G

In [6]:
#Алгоритм Флойда-Уоршелла
#Работает корректно, если в графе нет циклов отрицательной величины, 
#а в случае, когда такой цикл есть, позволяет найти хотя бы один такой цикл

#Сложность
#O(|V^3|")

#Находит кратчайший путь от всех пар вершин
#Негативные веса в ребрах разрешаются

#заведомо нет отрицательных циклов

def floyd_warshall(G, start, end):
    start -= 1
    end -= 1
    W = make_adjacency_matrix(G)
    for i in range(len(W)):
        for j in range(len(W[i])):
            if W[i][j] == 0 and i != j:
                W[i][j] = math.inf
    n = len(W)
    A = [[W[i][j] for i in range(n)] for j in range(n)] 
    V = [[math.inf for i in range(n)] for j in range(n)]
    for k in range(n): 
        for i in range(n):
            for j in range(n): 
                if A[i][j] > (A[i][k] + A[k][j]):
                    A[i][j] = A[i][k] + A[k][j]
                    V[i][j] = k+1
    path = [end+1, V[start][end]]
    if path[-1] != math.inf:
        while end != start+1:
            end = V[start][end]-1
            path.append(end)
    else:
        path.pop()
        path.append(start+1)
    return path[::-1]

In [8]:
G = make_graph(read_matrix("C:/Users/Goga/Desktop/Struct and algo/lab3/sources/notsymmatrix.txt"), 1)

res = floyd_warshall(G, 1, 6)
print(res)

new_graph(G, res).show("basic.html")


[[0, inf, inf, inf, inf, inf], [3, 0, inf, inf, 1, inf], [inf, 2, 1, 1, inf, inf], [inf, inf, 1, 0, inf, inf], [3, 1, inf, 1, 1, inf], [inf, inf, inf, 3, inf, 0]]


KeyError: 'weight'